# Interview Code Demonstration

## Ready-to-Run Examples for Technical Interviews

Quick demonstrations of key quant skills for live coding interviews.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
np.random.seed(42)

print("Interview Code Demo - Ready")

## Q1: Black-Scholes Option Pricing

**Interview Question**: Implement Black-Scholes call option pricing from scratch.

In [ ]:
# Black-Scholes Implementation
def black_scholes_call(S, K, T, r, sigma):
    """
    Calculate Black-Scholes call option price.
    
    S: Current stock price
    K: Strike price
    T: Time to expiration (years)
    r: Risk-free rate
    sigma: Volatility
    """
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    call_price = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    return call_price

# Example
price = black_scholes_call(S=100, K=105, T=0.5, r=0.05, sigma=0.2)
print(f"Call Option Price: ${price:.2f}")

## Q2: Sharpe Ratio Calculation

**Interview Question**: Calculate Sharpe ratio from returns.

In [ ]:
# Sharpe Ratio
def calculate_sharpe(returns, risk_free_rate=0.02, periods_per_year=252):
    """
    Calculate annualized Sharpe ratio.
    
    returns: Array of daily returns
    risk_free_rate: Annual risk-free rate
    periods_per_year: Trading days per year
    """
    # Convert daily risk-free to match return frequency
    daily_rf = risk_free_rate / periods_per_year
    
    # Excess returns
    excess_returns = returns - daily_rf
    
    # Annualize
    sharpe = np.sqrt(periods_per_year) * (excess_returns.mean() / excess_returns.std())
    return sharpe

# Example
daily_returns = np.random.normal(0.0005, 0.015, 252)  # Simulated returns
sharpe = calculate_sharpe(daily_returns)
print(f"Sharpe Ratio: {sharpe:.2f}")

## Q3: Value at Risk (VaR)

**Interview Question**: Calculate parametric and historical VaR.

In [ ]:
# Value at Risk
def calculate_var(returns, confidence=0.95, method='historical'):
    """
    Calculate Value at Risk.
    
    returns: Array of returns
    confidence: Confidence level (0.95 for 95%)
    method: 'historical' or 'parametric'
    """
    if method == 'historical':
        var = np.percentile(returns, (1 - confidence) * 100)
    else:  # parametric
        z_score = norm.ppf(1 - confidence)
        var = returns.mean() + z_score * returns.std()
    
    return var

# Example
returns = np.random.normal(0.0001, 0.02, 1000)
hist_var = calculate_var(returns, method='historical')
param_var = calculate_var(returns, method='parametric')

print(f"Historical VaR (95%): {hist_var:.4f}")
print(f"Parametric VaR (95%): {param_var:.4f}")

## Q4: Rolling Beta Calculation

**Interview Question**: Calculate rolling beta of a stock vs market.

In [ ]:
# Rolling Beta
def calculate_beta(stock_returns, market_returns, window=60):
    """
    Calculate rolling beta.
    
    stock_returns: Stock return series
    market_returns: Market return series
    window: Rolling window size
    """
    # Covariance and variance
    cov = pd.Series(stock_returns).rolling(window).cov(pd.Series(market_returns))
    var = pd.Series(market_returns).rolling(window).var()
    
    beta = cov / var
    return beta

# Example
market = np.random.normal(0.0004, 0.01, 252)
stock = 1.2 * market + np.random.normal(0, 0.005, 252)  # Beta ~1.2

beta = calculate_beta(stock, market, window=60)
print(f"Average Beta (60-day): {beta.iloc[-1]:.2f}")

## Q5: Maximum Drawdown

**Interview Question**: Calculate maximum drawdown from prices.

In [ ]:
# Maximum Drawdown
def calculate_max_drawdown(prices):
    """
    Calculate maximum drawdown.
    
    prices: Array or Series of prices
    """
    prices = np.array(prices)
    
    # Running maximum
    running_max = np.maximum.accumulate(prices)
    
    # Drawdown at each point
    drawdown = (prices - running_max) / running_max
    
    # Maximum drawdown
    max_dd = drawdown.min()
    max_dd_idx = drawdown.argmin()
    
    return max_dd, max_dd_idx

# Example
prices = 100 * np.cumprod(1 + np.random.normal(0.0002, 0.015, 252))
max_dd, dd_idx = calculate_max_drawdown(prices)

print(f"Maximum Drawdown: {max_dd:.2%}")
print(f"Occurred at day: {dd_idx}")

## Q6: Simple Moving Average Crossover

**Interview Question**: Implement SMA crossover trading signal.

In [ ]:
# SMA Crossover Signal
def sma_crossover_signal(prices, short_window=20, long_window=50):
    """
    Generate SMA crossover trading signals.
    
    Returns: 1 (buy), -1 (sell), 0 (hold)
    """
    prices = pd.Series(prices)
    
    short_sma = prices.rolling(short_window).mean()
    long_sma = prices.rolling(long_window).mean()
    
    # Signal: 1 when short > long (bullish), -1 when short < long (bearish)
    signal = np.where(short_sma > long_sma, 1, -1)
    signal = pd.Series(signal, index=prices.index)
    
    # Trade signals (crossovers only)
    trade_signal = signal.diff()
    
    return signal, trade_signal

# Example
prices = 100 * np.cumprod(1 + np.random.normal(0.0003, 0.02, 252))
signal, trades = sma_crossover_signal(prices)

n_trades = (trades != 0).sum()
print(f"Number of trades: {n_trades}")
print(f"Current signal: {'Long' if signal.iloc[-1] > 0 else 'Short'}")

## Q7: Portfolio Variance

**Interview Question**: Calculate portfolio variance given weights and covariance.

In [ ]:
# Portfolio Variance
def portfolio_variance(weights, cov_matrix):
    """
    Calculate portfolio variance.
    
    weights: Array of portfolio weights
    cov_matrix: Covariance matrix of returns
    """
    weights = np.array(weights)
    variance = weights.T @ cov_matrix @ weights
    return variance

# Example: 3-asset portfolio
weights = np.array([0.4, 0.35, 0.25])  # Sum to 1

# Correlation and volatilities -> covariance
vols = np.array([0.15, 0.20, 0.25])  # Annualized
corr = np.array([
    [1.0, 0.3, 0.2],
    [0.3, 1.0, 0.5],
    [0.2, 0.5, 1.0]
])
cov_matrix = np.outer(vols, vols) * corr

port_var = portfolio_variance(weights, cov_matrix)
port_vol = np.sqrt(port_var)

print(f"Portfolio Variance: {port_var:.4f}")
print(f"Portfolio Volatility: {port_vol:.2%}")

## 🎯 Interview Tips

1. **Explain as you code** - Walk through your logic
2. **Start simple** - Get working solution, then optimize
3. **Handle edge cases** - Division by zero, empty arrays
4. **Know complexity** - Time and space complexity
5. **Test your code** - Run with example inputs